# Orquestrador de Chatbots - Etapa 5 (Final)
-----------------------------------
## Testes os classificadores Voting finais com as bases de validação

1. Carregar os classificadores Voting gerados anteriormente.
2. Testar os classificadores finais com a base de validação, sem as perguntas fora de contexto.
5. Testar os classificadores finais com a base de validação, com as perguntas fora de contexto.
6. Incluir solução para classificar pelo menos algumas das perguntas fora de contexto e analisar os resultados.

In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Bibliotecas utilizadas

In [48]:
import pandas as pd
import numpy as np
import csv
import codecs
import os
import glob
import pickle
import re
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, make_scorer

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV

from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import json

from datetime import datetime

In [49]:
import orquestrador_funcoes_gerais as ofg

## Funções Específicas

In [50]:
def classificar(df, bots, vectorizer, verbose=True):
    
    if verbose:
        print('Classificando', len(df), 'perguntas\n')
    X_test = vectorizer.transform(df['pergunta'].tolist())
    
    for bot in bots:
        if verbose:
            print('Classificando com classificador',bot['nome'])
        y_transform = bot['clf'].transform(X_test)
        y_soma = []
        for y in y_transform:
            # Obtém a média ponderada (pelos pesos) das probabilidades de "1"
            soma_ponderada = 0
            for index in range(len(bot['clf'].weights)):
                soma_ponderada += bot['clf'].weights[index] * (y[(index*2)+1])
            y_soma.append(soma_ponderada/sum(bot['clf'].weights))
        df['proba_' + str(bot['bot_id'])] = y_soma 
        
    return calcular_classes(df, bots)   

In [51]:
def calcular_classes(df, bots):
    df['bot_id_previsto'] = 0
    for index, row in df.iterrows():
        maximo = -1
        bot_id_maximo = 0
        probas = []
        for bot in bots:
            proba_bot = row['proba_' + str(bot['bot_id'])]
            probas.append(proba_bot)
            # A classe escolhida será sempre a que tiver maior probabilidade de rótulo "1"
            if (proba_bot > maximo):
                maximo = proba_bot
                bot_id_maximo = bot['bot_id']
        df.at[index,'bot_id_previsto'] = bot_id_maximo         
    return df  

In [52]:
def conferencia_validacao(df, modalidade='', modo=1):
    if modalidade == '!=0':
        y_test =  df[df['bot_id_verdade'] != 0]['bot_id_verdade'].tolist()
        y_predicted = df[df['bot_id_verdade'] != 0]['bot_id_previsto'].tolist()
        titulo = 'Somente Registros com bot_id != 0'
    elif modalidade == '==0':
        y_test =  df[df['bot_id_verdade'] == 0]['bot_id_verdade'].tolist()
        y_predicted = df[df['bot_id_verdade'] == 0]['bot_id_previsto'].tolist()
        titulo = 'Somente Registros com bot_id == 0'
    else:
        y_test =  df['bot_id_verdade'].tolist()
        y_predicted =  df['bot_id_previsto'].tolist()
        titulo = 'Todos os Registros'
    
    if modo==1:
        print('Métricas de Desempenho - ' + titulo + ':')
        print('Total de Registros Considerados:',len(y_test),'(',round(len(y_test)*100/len(df),4),'%)')
        accuracy, precision, recall, f1 = ofg.get_metrics(y_test, y_predicted) 
        print('-'*120)
    else:
        return ofg.get_metrics(y_test, y_predicted, verbose=False)  

In [53]:
# Método para classificar uma pergunta fora de contexto...
# mpfv = Máximo de Palavras Fora do Dicionário
# mpco = Mínimo Probabilidade da Classe Original

def classificar_0(df, bots, vocabulario, mpfv, mpco, verbose=True):
    if vocabulario == []:
        return df
    
    df = classificar(df, bots, vectorizer, verbose)
    
    for index, row in df.iterrows():
        
        # Se a probabilidade da classe original for acima do limite mínimo de confiança, desconsidera a pergunta.
        if df.at[index,'proba_' + str(df.at[index,'bot_id_previsto'])] > mpco:
            continue
 
        # Verifica quantas palavras da pergunta estão fora do vocabulário...
        tokens = ofg.tokenizer.tokenize(df.at[index,'pergunta'])
        total_palavras = len(tokens)
        tokens = [palavra for palavra in tokens if palavra not in vocabulario] 

        # ... se for acima do limite especificado, determina que a pergunta é fora de contexto.
        if len(tokens) > mpfv:
            df.at[index,'bot_id_previsto'] = 0

    return df

## Configurações

In [54]:
cfg = ofg.carregar_configuracoes()

Conferir as configurações antes de prosseguir
nome_arquivo_configuracao: config.json
------------------------------------------------------------------------------------------------------------------------
aplicar_stemmer: False
------------------------------------------------------------------------------------------------------------------------
bots: [{'bot_id': 1, 'nome': 'Alistamento Militar', 'arquivo': 'skill-alistamento-militar.json'}, {'bot_id': 2, 'nome': 'COVID', 'arquivo': 'skill-covid.json'}, {'bot_id': 3, 'nome': 'Login Único', 'arquivo': 'skill-login-unico.json'}, {'bot_id': 4, 'nome': 'IRPF 2020', 'arquivo': 'skill-perguntao-irpf-2020.json'}, {'bot_id': 5, 'nome': 'PGMEI', 'arquivo': 'skill-pgmei.json'}, {'bot_id': 6, 'nome': 'Selo Turismo Responsável', 'arquivo': 'skill-poc-selo-turismo-responsavel.json'}, {'bot_id': 7, 'nome': 'Cadastur', 'arquivo': 'skill-cadastur.json'}, {'bot_id': 8, 'nome': 'Tuberculose', 'arquivo': 'skill-tuberculose.json'}]
---------------------

In [55]:
bots=cfg['bots']

In [56]:
print('ATENÇÃO!!! Aplicação de Stemmer =', cfg['aplicar_stemmer'])

ATENÇÃO!!! Aplicação de Stemmer = False


## Carregar Vetorizador

In [57]:
arquivo_vetorizador = os.path.join(os.getcwd(), cfg['diretorio_modelos'], cfg['arquivo_vetorizador'])
print('Carregando Vetorizador --->',arquivo_vetorizador,'\n')
try:            
    file = open(arquivo_vetorizador, 'rb')
    vectorizer = pickle.load(file)
    file.close()
except Exception as e:
    print('Erro no carregamento do vetorizador',arquivo_vetorizador,'-->',str(e))
print(vectorizer)

Carregando Vetorizador ---> E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\modelos\vetorizador.pkl 

TfidfVectorizer(ngram_range=[1, 2], smooth_idf=False, sublinear_tf=True)


## Carregar Classificadores Voting por bot

In [58]:
print('Carregando os classificadores Voting por classe')
print('-'*120)
clfs_base = []
padrao_arquivo_classificador_voting = os.path.join(os.getcwd(), cfg['diretorio_modelos'],
                                                   cfg['padrao_arquivo_classificador_voting'])
for bot in bots:
    arquivo_classificador_voting = padrao_arquivo_classificador_voting.replace('%bot_id%',str(bot['bot_id']))
    try:            
        file = open(arquivo_classificador_voting, 'rb')
        bot['clf'] = pickle.load(file)
        file.close()
        print('Carregado:','%-30s' % (str(bot['bot_id']) + ' - ' + bot['nome']),arquivo_classificador_voting)
    except Exception as e:
        print('Erro no carregamento do modelo',arquivo_classificador_voting,'-->',str(e))
print('-'*120)            

Carregando os classificadores Voting por classe
------------------------------------------------------------------------------------------------------------------------
Carregado: 1 - Alistamento Militar        E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\modelos\clf_voting_1.pkl
Carregado: 2 - COVID                      E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\modelos\clf_voting_2.pkl
Carregado: 3 - Login Único                E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\modelos\clf_voting_3.pkl
Carregado: 4 - IRPF 2020                  E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\modelos\clf_voting_4.pkl
Carregado: 5 - PGMEI                      E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\modelos\clf_voting_5.pkl
Carregado: 6 - Selo Turismo Responsável   E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\modelos\clf_voting_6.pkl
Carregado: 7 - Cadastur                   E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\modelos\clf_voting_7.pkl
Carre

## Testa classificadores com a base de validação, SEM perguntas fora de contexto

In [59]:
arquivo_validacao = os.path.join(os.getcwd(),  cfg['diretorio_dados'], cfg['arquivo_validacao']) 
df_val = pd.read_csv(arquivo_validacao, index_col=None, engine='python', sep =',', encoding="utf-8")

df_val = df_val.rename(columns={"bot_id": "bot_id_verdade"}, errors="raise")

print('Total de perguntas da base de validação (sem 0):',len(df_val), 'de', arquivo_validacao)
df_val.head(10)

Total de perguntas da base de validação (sem 0): 1428 de E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\dados\validacao.csv


,bot_id_verdade,pergunta
0,6,me diga o que é protocolo
1,1,CTSM
2,4,alzhimer deduz
3,4,irpf PAGO INDEVIDAMENTE
4,3,Eu não criei essa conta
5,2,Como fazer álcool em gel caseiro?
6,6,HAVERA VARIAÇÃO NAS TAXAS DE SERVIÇOS DEVIDO A...
7,1,Esqueci de me alistar o que faço?
8,4,Sou obrigado a apresentar declaração de impost...
9,4,tenho como entregar a declaração pelo celular?


In [60]:
# Distribuição das classes. Note que não há perguntas fora de contexto (id = 0)
df_val.groupby('bot_id_verdade').count()

,pergunta
bot_id_verdade,
1,128
2,62
3,157
4,648
5,39
6,210
7,103
8,81


In [61]:
# Processa as perguntas, fazendo a limpeza dos textos
df_val['pergunta'] = df_val['pergunta'].apply(lambda x: ofg.limpar_texto(x, cfg['aplicar_stemmer']))
df_val

,bot_id_verdade,pergunta
0,6,diga protocolo
1,1,certidao tempo servico militar
2,4,alzhimer deduz
3,4,irpf pago indevidamente
4,3,nao criei conta
...,...,...
1423,7,cnae
1424,4,declarar recebimento seguro carro
1425,8,existe tuberculose olhos
1426,7,transportadora turistica


In [62]:
# Confere se não ocorreu alguma pergunta vazia
# Numa situação de produção, essas perguntas ficariam com bot_id = 0
df_val.loc[np.array(list(map(len,df_val.pergunta)))==0]

,bot_id_verdade,pergunta


In [63]:
# Faz a classificação das perguntas
df_val = classificar(df_val, bots, vectorizer)
df_val

Classificando 1428 perguntas

Classificando com classificador Alistamento Militar
Classificando com classificador COVID
Classificando com classificador Login Único
Classificando com classificador IRPF 2020
Classificando com classificador PGMEI
Classificando com classificador Selo Turismo Responsável
Classificando com classificador Cadastur
Classificando com classificador Tuberculose


,bot_id_verdade,pergunta,proba_1,proba_2,proba_3,proba_4,proba_5,proba_6,proba_7,proba_8,bot_id_previsto
0,6,diga protocolo,0.012787,0.013671,0.015598,0.023947,0.017078,0.915317,0.022619,0.027548,6
1,1,certidao tempo servico militar,0.999666,0.007993,0.005063,0.002245,0.005920,0.008713,0.009864,0.003601,1
2,4,alzhimer deduz,0.008205,0.008920,0.011206,0.963919,0.014353,0.017793,0.022067,0.004764,4
3,4,irpf pago indevidamente,0.008323,0.010614,0.013110,0.920328,0.009781,0.034418,0.074564,0.011872,4
4,3,nao criei conta,0.019766,0.013916,0.928249,0.034141,0.017188,0.048579,0.013535,0.005349,3
...,...,...,...,...,...,...,...,...,...,...,...
1423,7,cnae,0.015814,0.013161,0.020137,0.042797,0.021388,0.025711,0.840676,0.006516,7
1424,4,declarar recebimento seguro carro,0.007893,0.016713,0.057607,0.961107,0.008236,0.029594,0.016436,0.004957,4
1425,8,existe tuberculose olhos,0.008806,0.010850,0.014867,0.010276,0.010444,0.020726,0.027037,0.988208,8
1426,7,transportadora turistica,0.024663,0.021750,0.034421,0.078623,0.024675,0.084792,0.294608,0.009131,7


In [64]:
# Resultados
conferencia_validacao(df_val)

Métricas de Desempenho - Todos os Registros:
Total de Registros Considerados: 1428 ( 100.0 %)
Acurácia: 0.9244 - Precisão (Macro): 0.9175 - Recall (Macro): 0.8776 - F1 (Macro): 0.8959 - F1 (Weighted): 0.923
------------------------------------------------------------------------------------------------------------------------


## Testa classificadores com a base de validação, **COM** perguntas fora de contexto

In [65]:
# Carrega as perguntas não rotuladas da base de validação
arquivo_validacao_com_zero = os.path.join(os.getcwd(),  cfg['diretorio_dados'], cfg['arquivo_validacao_com_zero']) 
df_val_0 = pd.read_csv(arquivo_validacao_com_zero, index_col=None, engine='python', sep =',', encoding="utf-8")

df_val_0 = df_val_0.rename(columns={"bot_id": "bot_id_verdade"}, errors="raise")

print('Total de perguntas da base de validação (com 0):',len(df_val_0), 'de', arquivo_validacao_com_zero)
df_val_0.head(10)

Total de perguntas da base de validação (com 0): 1683 de E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\dados\validacao_com_0.csv


,bot_id_verdade,pergunta
0,6,O LOCAL ONDE ME HOSPEDEI TEM O SELO MAS ESTA C...
1,7,ouvidoria
2,6,é obrigatorio
3,4,posso receber a restituição do IRPF no primeir...
4,6,qual o protocolo a ser seguido pelo hoteis
5,8,quero saber sobre tuberculose ganglionar
6,3,Com o login único eu acesso quais serviços ?
7,1,Qual a documentação necessária para se dar ent...
8,4,como faço para corrigir o erro se apurei ganho...
9,4,declarar rendimentos mesmo sem ser obrigado?


In [66]:
# Distribuição das classes. Note que agora HÁ perguntas fora de contexto (id = 0)
df_val_0.groupby('bot_id_verdade').count()

,pergunta
bot_id_verdade,
0,255
1,128
2,62
3,157
4,648
5,39
6,210
7,103
8,81


In [67]:
# Alguns exemplos de pergunta fora de contexto
df_val_0[df_val_0['bot_id_verdade']==0]

,bot_id_verdade,pergunta
14,0,Como Atribuir o Selo Certificado Digital de Pe...
16,0,O que é pensão por morte?
25,0,Quais são as principais formas de transmissão ...
30,0,O que é substituição tributária?
35,0,O que é Meu INSS?
...,...,...
1655,0,Como o eleitor portador de deficiência deve pr...
1672,0,Algum outro mosquito é capaz de transmitir a d...
1673,0,Como Retiro Colaborador do CNPJ?
1679,0,"Ansiedade em crianças, é possível?"


In [68]:
# Processa as perguntas, fazendo a limpeza dos textos
df_val_0['pergunta'] = df_val_0['pergunta'].apply(lambda x: ofg.limpar_texto(x, cfg['aplicar_stemmer']))
df_val_0

,bot_id_verdade,pergunta
0,6,local onde hospedei selo cheio irregularidade ...
1,7,ouvidoria
2,6,obrigatorio
3,4,posso receber restituicao irpf primeiro lote
4,6,protocolo ser seguido hoteis
...,...,...
1678,4,ate dia entregar declaracao imposto renda pess...
1679,0,ansiedade criancas possivel
1680,4,despesas paciente cancer deduz
1681,3,pedem complementar dados mail telefone login u...


In [69]:
# Confere se não ocorreu alguma pergunta vazia.
# Numa situação de produção, essas perguntas ficariam com bot_id = 0
df_val_0.loc[np.array(list(map(len,df_val_0.pergunta)))==0]

,bot_id_verdade,pergunta


In [70]:
# Faz a classificação das perguntas
df_val_0 = classificar(df_val_0, bots, vectorizer)
df_val_0

Classificando 1683 perguntas

Classificando com classificador Alistamento Militar
Classificando com classificador COVID
Classificando com classificador Login Único
Classificando com classificador IRPF 2020
Classificando com classificador PGMEI
Classificando com classificador Selo Turismo Responsável
Classificando com classificador Cadastur
Classificando com classificador Tuberculose


,bot_id_verdade,pergunta,proba_1,proba_2,proba_3,proba_4,proba_5,proba_6,proba_7,proba_8,bot_id_previsto
0,6,local onde hospedei selo cheio irregularidade ...,0.020775,0.019198,0.022673,0.020603,0.021758,0.783091,0.033777,0.005390,6
1,7,ouvidoria,0.014524,0.013885,0.015903,0.032307,0.018460,0.817263,0.372256,0.006821,6
2,6,obrigatorio,0.055913,0.016650,0.037850,0.117124,0.019557,0.247926,0.117282,0.006949,6
3,4,posso receber restituicao irpf primeiro lote,0.004101,0.008784,0.001968,0.996799,0.005686,0.009491,0.016547,0.003396,4
4,6,protocolo ser seguido hoteis,0.011047,0.025406,0.005270,0.022503,0.008510,0.975756,0.018405,0.015863,6
...,...,...,...,...,...,...,...,...,...,...,...
1678,4,ate dia entregar declaracao imposto renda pess...,0.008454,0.006312,0.002668,0.998509,0.004017,0.005886,0.007298,0.001801,4
1679,0,ansiedade criancas possivel,0.058926,0.024978,0.123794,0.403772,0.021216,0.044549,0.034470,0.008192,4
1680,4,despesas paciente cancer deduz,0.005211,0.008399,0.006527,0.993911,0.010126,0.016085,0.013290,0.004650,4
1681,3,pedem complementar dados mail telefone login u...,0.008148,0.005665,0.993916,0.011265,0.006594,0.015659,0.010139,0.004370,3


In [71]:
# Resultados. Agora as métricas devem cair bastante, pois não há tratamento das perguntas 0, que são parte considerável da 
# base. Todos esses registros estarão errados, e esses erros se somarão ao erros normais. Portanto, um tratamento tem que ser
# dado a esses registros.
conferencia_validacao(df_val_0)

Métricas de Desempenho - Todos os Registros:
Total de Registros Considerados: 1683 ( 100.0 %)
Acurácia: 0.7843 - Precisão (Macro): 0.6851 - Recall (Macro): 0.7801 - F1 (Macro): 0.7281 - F1 (Weighted): 0.7192
------------------------------------------------------------------------------------------------------------------------


## Tratamento das perguntas fora de contexto ("Perguntas Zero")

Como se viu acima, os classificadores estão fazendo um bom trabalho ao identificar corretamente as classes das perguntas que estão dentro do contexto dos bots abordados. Mas as perguntas fora de contexto, ou seja, aquelas que não se destinam a nenhum bot, e que são um percentual considerável da base de validação, puxam as métricas totais para baixo.

Não há como treinar um classificador para identificar esses  "casos zero" pq essas perguntas são **imprevisíveis**. O conjunto de "perguntas zero" que veio nesse arquivo de validação vale apenas para o presente momento e não são úteis para se construir um classificador. 

A única solução é tentar encontrar algum critério matemático. Por exemplo:

1. Se nenhum dos classificadores considerar a pergunta como pertencente ao "seu" bot

2. Se nenhuma probabilidade retornada for acima de um certo valor

3. Se há "indecisão" em excesso, ou seja, se mais de X classificadores indicaram que a pergunta pertence ao "seu bot".

4. Analisar se é possível separar as perguntas fora de contexto pela distribuição estatística das perguntas dentro do contexto.

5. Se as perguntas em questão possuem um certo número de palavras fora do vocabulário

6. Outro...?

## A melhor solução até agora...

Após várias experiências, a melhor solução foi: somar o número de palavras fora do vocabulário de um pergunta. Se existir um número maior do que um limite, determina que a pergunta é fora de contexto, **desde que a probabilidade da classe definida originalmente seja menor que um determinado limite**.

Os limites podem ser definidos de forma mais conservadora ou agressiva. Uma configuração mais agressiva fará com que muitas perguntas que poderiam ser direcionadas para um bot serem consideradas fora de contexto, exigindo que o usuário refaça melhor tais perguntas. Qual configuração é a mais adequada dependerá do modelo de negócio e das prioridades definidas pelo cliente. Para exemplificar isso, são mostradas as parametrizações com foco na performance geral, e com foco na performance das classes de contexto.

Apesar de algumas das melhores métricas levarem em consideração apenas a probabilidade, elas tendem a levar a mais falsos positivos da classe 0. O ideal é procurar por limites que resultem no melhor trade-off para as prioridades de negócio.

In [72]:
# Carrega vocabulário
vocabulario = []
arquivo_vocab = os.path.join(os.getcwd(), cfg['diretorio_dados'], cfg['arquivo_vocabulario'])
print('Carregando vocabulário de', arquivo_vocab,'\n')
try:
    with open(arquivo_vocab, "r") as arq_vocab:
        for p in arq_vocab:
            vocabulario.append(p.strip())
    print('Vocabulário:', len(vocabulario),'palavras.') 
except:
    print('Não encontrou',arquivo_vocab)

Carregando vocabulário de E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\dados\vocabulario.txt 

Vocabulário: 2135 palavras.


In [73]:
# Faz a classificação das perguntas, agora tentando encontrar as perguntas fora de contexto.
# Quanto mais agressivas as configurações, mais perguntas serão consideradas como fora de contexto, e mais erros de
# perguntas classificadas erroneamente serão obtidos. Mas isso pode ser interessante, dependendo das prioridades do cliente.
maximo_palavras_fora_dicionario = 2
minimo_probabilidade_classe = 0.57
df_val_0 = classificar_0(df_val_0, bots, vocabulario, maximo_palavras_fora_dicionario, minimo_probabilidade_classe)

Classificando 1683 perguntas

Classificando com classificador Alistamento Militar
Classificando com classificador COVID
Classificando com classificador Login Único
Classificando com classificador IRPF 2020
Classificando com classificador PGMEI
Classificando com classificador Selo Turismo Responsável
Classificando com classificador Cadastur
Classificando com classificador Tuberculose


In [74]:
# Resultados. As métricas não serão tão boas quanto em uma base somente de perguntas contextuais, mas devem ser 
# razoavelmente superiores à última tentativa.
conferencia_validacao(df_val_0, '')
conferencia_validacao(df_val_0, '!=0')
conferencia_validacao(df_val_0, '==0')
df_0 = df_val_0[df_val_0['bot_id_previsto'] == 0]
print('\nTotal de Perguntas Classificadas como fora de contexto:',len(df_0))
df_0

Métricas de Desempenho - Todos os Registros:
Total de Registros Considerados: 1683 ( 100.0 %)
Acurácia: 0.814 - Precisão (Macro): 0.8177 - Recall (Macro): 0.8019 - F1 (Macro): 0.7801 - F1 (Weighted): 0.7806
------------------------------------------------------------------------------------------------------------------------
Métricas de Desempenho - Somente Registros com bot_id != 0:
Total de Registros Considerados: 1428 ( 84.8485 %)
Acurácia: 0.9244 - Precisão (Macro): 0.8166 - Recall (Macro): 0.7801 - F1 (Macro): 0.7969 - F1 (Weighted): 0.924
------------------------------------------------------------------------------------------------------------------------
Métricas de Desempenho - Somente Registros com bot_id == 0:
Total de Registros Considerados: 255 ( 15.1515 %)
Acurácia: 0.1961 - Precisão (Macro): 0.125 - Recall (Macro): 0.0245 - F1 (Macro): 0.041 - F1 (Weighted): 0.3279
---------------------------------------------------------------------------------------------------------

,bot_id_verdade,pergunta,proba_1,proba_2,proba_3,proba_4,proba_5,proba_6,proba_7,proba_8,bot_id_previsto
38,0,atribuir selo cadastro basico validacao dados ...,0.006970,0.004363,0.537483,0.001807,0.006482,0.438403,0.424459,0.003548,0
78,0,funcionam autotestes deteccao hiv,0.021934,0.548834,0.033649,0.084180,0.024245,0.046454,0.046567,0.007606,0
119,0,incidencia hipertensao aumentando publico jovem,0.018453,0.027665,0.292349,0.190758,0.025151,0.051242,0.086322,0.007447,0
209,0,partidos politicos podem fiscalizar votacao ap...,0.021395,0.079140,0.016776,0.346614,0.017057,0.065929,0.116621,0.017469,0
223,0,pessoa reconhece mosquito aedes aegypti,0.151432,0.289261,0.031146,0.115474,0.017312,0.028781,0.030583,0.021874,0
288,0,acordo segunda topica freudiana exaltado organ...,0.236641,0.050665,0.023361,0.103533,0.022668,0.055187,0.041002,0.008548,0
331,0,principal mosquito transmissor dengue,0.021835,0.020954,0.030566,0.074781,0.524983,0.044496,0.024402,0.009033,0
356,0,quanto tempo posso dirigir cnh vencida,0.116755,0.082904,0.010473,0.073994,0.005903,0.531862,0.078405,0.003455,0
430,0,assim surgimento comercio virtual estende alca...,0.024102,0.023244,0.037518,0.417326,0.025687,0.073454,0.046007,0.020657,0
447,0,posso ter certeza nao ha votos registrados urn...,0.025992,0.029645,0.454786,0.081084,0.010508,0.135202,0.050195,0.007184,0


In [75]:
# Uma análise das perguntas consideradas errôneamente fora de contexto. Nota-se que várias delas são ambíguas mesmo, o que 
# torna interessante classificá-las como zero, daí exigindo um texto melhor do usuário
df_err = df_val_0[(df_val_0['bot_id_previsto'] == 0) & (df_val_0['bot_id_verdade'] != 0)]
perc_err = round(len(df_err) * 100/len(df_val_0),2)
print(len(df_err),'(' + ('%2.2f' % perc_err) + '% do total) perguntas errôneamente classificadas como fora de contexto.')
df_err 

3 (0.18% do total) perguntas errôneamente classificadas como fora de contexto.


,bot_id_verdade,pergunta,proba_1,proba_2,proba_3,proba_4,proba_5,proba_6,proba_7,proba_8,bot_id_previsto
916,2,recebi encomenda china devo tomar cuidado,0.031540,0.454945,0.038688,0.563918,0.016742,0.030516,0.013716,0.009209,0
971,8,beijos tambem nao causam transmissao doenc,0.042735,0.024429,0.188051,0.077413,0.031667,0.115247,0.026288,0.027944,0
1144,2,goticulas espirro podem ficar suspensao contam...,0.019275,0.109397,0.016993,0.232074,0.012554,0.031240,0.107878,0.055808,0


In [76]:
colunas_scores = ['Parâmetros', 'Acurácia', 'Precisão', 'Recall','F1',
                   'F1(Sem 0)', 'F1(Só 0)', 'Total 0', 'Err 0', '% Err 0', 'Ok 0', '% Ok 0']

In [77]:
# Faz experimentos com os limites na tentativa de avaliar quais são, aproximadamente, os de melhor retorno.
total_0 = len(df_val_0[df_val_0 ['bot_id_verdade']==0])

maximo_palavras_fora_dicionario = list(range(0,6))
minimo_probabilidade_classe = np.arange(0.4, 0.93, 0.03)
#maximo_palavras_fora_dicionario = [2]
#minimo_probabilidade_classe = [0.61]
total_combinacoes = len(maximo_palavras_fora_dicionario) * len(minimo_probabilidade_classe)
index = 0

df_scores = pd.DataFrame(columns=colunas_scores)

print('Total Combinações:', total_combinacoes, 
      '--> Tempo aproximado de processamento:',round(total_combinacoes * 13.25 / 60, 2),'minutos. \n')

hora_inicio = datetime.now()

for mpfv in maximo_palavras_fora_dicionario:
    for mpco in minimo_probabilidade_classe:
        index+=1
        ofg.print_progress_bar(index,total_combinacoes,prefix="Processando:", suffix="Completo")   
        df_val_0 = classificar_0(df_val_0, bots, vocabulario, mpfv, mpco, verbose=False)
        accuracy_tudo, precision_tudo, recall_tudo, f1_tudo = conferencia_validacao(df_val_0, '', modo=2)
        accuracy, precision, recall, f1_sem0 = conferencia_validacao(df_val_0, '!=0', modo=2)
        accuracy, precision, recall, f1_soh0 = conferencia_validacao(df_val_0, '==0', modo=2)
        total_0 = len(df_val_0[df_val_0['bot_id_previsto'] == 0])
        df_err_0 = df_val_0[(df_val_0['bot_id_previsto'] == 0) & (df_val_0['bot_id_verdade'] != 0)]
        total_err_0 = len(df_err_0)
        perc_err_0 = total_err_0/total_0
        score = {'Parâmetros':"{'stemmer':%s, 'mpfv':%i, 'mpco':%.3f}" % (cfg['aplicar_stemmer'], mpfv, mpco), 
                 'Acurácia':round(accuracy_tudo,3), 'Precisão':round(precision_tudo,3), 
                 'Recall':round(recall_tudo,3), 'F1':round(f1_tudo,3), 'F1(Sem 0)':round(f1_sem0,3), 
                 'F1(Só 0)':round(f1_soh0,3), 'Total 0':round(total_0,3), 'Err 0':round(total_err_0,3), 
                 '% Err 0':round(perc_err_0,3), 'Ok 0':round(total_0 - total_err_0,3), '% Ok 0':round(1 - perc_err_0,3)}
        df_scores = df_scores.append(score,ignore_index=True)  

hora_fim = datetime.now()
intervalo = (hora_inicio - hora_fim).seconds / 60
print('Fim Processamento - tempo Total (min):', round(intervalo/60,2) , 
      '- Por Combinação (s):', round(intervalo/total_combinacoes,3), '\n')       
            
df_scores = df_scores.sort_values(by=['F1','F1(Sem 0)','F1(Só 0)'], ascending=False) 
df_scores = df_scores.reset_index(drop=True)   
df_scores.head(40)

Total Combinações: 108 --> Tempo aproximado de processamento: 23.85 minutos. 

Processando: |██████████████████████████████████████████████████| 100.0% Completo
Fim Processamento - tempo Total (min): 23.87 - Por Combinação (s): 13.261 



,Parâmetros,Acurácia,Precisão,Recall,F1,F1(Sem 0),F1(Só 0),Total 0,Err 0,% Err 0,Ok 0,% Ok 0
0,"{'stemmer':False, 'mpfv':0, 'mpco':0.760}",0.866,0.854,0.829,0.840,0.788,0.099,233,66,0.283,167,0.717
1,"{'stemmer':False, 'mpfv':0, 'mpco':0.730}",0.865,0.850,0.830,0.838,0.790,0.096,219,59,0.269,160,0.731
2,"{'stemmer':False, 'mpfv':0, 'mpco':0.700}",0.862,0.848,0.830,0.836,0.792,0.093,207,55,0.266,152,0.734
3,"{'stemmer':False, 'mpfv':0, 'mpco':0.790}",0.863,0.852,0.820,0.834,0.781,0.100,247,76,0.308,171,0.692
4,"{'stemmer':False, 'mpfv':0, 'mpco':0.820}",0.863,0.854,0.817,0.833,0.777,0.104,268,86,0.321,182,0.679
5,"{'stemmer':False, 'mpfv':0, 'mpco':0.670}",0.856,0.843,0.825,0.831,0.791,0.089,196,54,0.276,142,0.724
6,"{'stemmer':False, 'mpfv':0, 'mpco':0.640}",0.854,0.840,0.825,0.829,0.792,0.087,183,47,0.257,136,0.743
7,"{'stemmer':False, 'mpfv':1, 'mpco':0.880}",0.854,0.842,0.825,0.828,0.791,0.085,155,23,0.148,132,0.852
8,"{'stemmer':False, 'mpfv':0, 'mpco':0.850}",0.859,0.851,0.812,0.828,0.772,0.106,289,101,0.349,188,0.651
9,"{'stemmer':False, 'mpfv':0, 'mpco':0.880}",0.857,0.855,0.810,0.828,0.771,0.108,305,112,0.367,193,0.633


In [78]:
# Persiste o dataframe de resultados:
arquivo_resultados = os.path.join(os.getcwd(), cfg['diretorio_dados'], cfg['arquivo_resultados'][str(cfg['aplicar_stemmer'])])
with open(arquivo_resultados, "wb") as df_file:
    pickle.dump(df_scores, df_file)
print('Dataframe de resultados salvo em:', arquivo_resultados)

Dataframe de resultados salvo em: E:\DataScience\PUC\TCC\tcc_orquestrador_bots_final\dados\voting_resultados_sem_stemmer.pkl


## Carrega todos os resultados obtidos e ordena num dataframe final

In [79]:
arquivo_vetorizador = os.path.join(os.getcwd(), cfg['diretorio_dados'], cfg['arquivo_vetorizador'])
print('Carregando Resultados...')
try:            
    file = open(os.path.join(os.getcwd(), cfg['diretorio_dados'], cfg['arquivo_resultados']['True']), 'rb')
    df_results_cs = pickle.load(file)
    file.close()
    
    file = open(os.path.join(os.getcwd(), cfg['diretorio_dados'], cfg['arquivo_resultados']['False']), 'rb')
    df_results_ss = pickle.load(file)
    file.close()
    
except Exception as e:
    print('Erro no carregamento dos resultados -->',str(e))

Carregando Resultados...


In [80]:
df_results_all = pd.concat([df_results_cs, df_results_ss])

In [81]:
# Isso é só para corrigir um erro meu, pode retirar depois
if False:
    for index, row in df_results_all.iterrows():
        par = df_results_all.at[index,'Parâmetros']
        df_results_all.at[index,'Parâmetros'] = par.replace('mpfd','mpfv').replace('mpc','mpco')            

In [82]:
# Foco na maior performance geral
ordenacao = ['F1','F1(Sem 0)','F1(Só 0)','Acurácia']
df_results_all = df_results_all.sort_values(by=ordenacao, ascending=False) 
df_results_all = df_results_all.reset_index(drop=True)
print('\tScores Finais, por', ordenacao, '(Obs: Total Perguntas Zero = %i)' % len(df_val_0[df_val_0 ['bot_id_verdade']==0]))
df_results_all.head(40)    

	Scores Finais, por ['F1', 'F1(Sem 0)', 'F1(Só 0)', 'Acurácia'] (Obs: Total Perguntas Zero = 255)


,Parâmetros,Acurácia,Precisão,Recall,F1,F1(Sem 0),F1(Só 0),Total 0,Err 0,% Err 0,Ok 0,% Ok 0
0,"{'stemmer':True, 'mpfv':0, 'mpco':0.790}",0.875,0.870,0.842,0.854,0.800,0.087,207,44,0.213,163,0.787
1,"{'stemmer':True, 'mpfv':0, 'mpco':0.820}",0.876,0.870,0.841,0.854,0.798,0.088,213,46,0.216,167,0.784
2,"{'stemmer':True, 'mpfv':0, 'mpco':0.850}",0.878,0.870,0.837,0.852,0.794,0.091,232,56,0.241,176,0.759
3,"{'stemmer':True, 'mpfv':0, 'mpco':0.760}",0.872,0.867,0.840,0.851,0.800,0.084,199,43,0.216,156,0.784
4,"{'stemmer':True, 'mpfv':0, 'mpco':0.730}",0.868,0.864,0.840,0.849,0.802,0.081,187,40,0.214,147,0.786
5,"{'stemmer':True, 'mpfv':0, 'mpco':0.700}",0.865,0.861,0.837,0.846,0.802,0.080,182,40,0.220,142,0.780
6,"{'stemmer':True, 'mpfv':0, 'mpco':0.880}",0.872,0.868,0.829,0.846,0.789,0.092,252,72,0.286,180,0.714
7,"{'stemmer':True, 'mpfv':0, 'mpco':0.670}",0.860,0.855,0.835,0.842,0.803,0.076,172,39,0.227,133,0.773
8,"{'stemmer':False, 'mpfv':0, 'mpco':0.760}",0.866,0.854,0.829,0.840,0.788,0.099,233,66,0.283,167,0.717
9,"{'stemmer':True, 'mpfv':0, 'mpco':0.910}",0.870,0.865,0.820,0.840,0.781,0.094,268,82,0.306,186,0.694


In [83]:
# Se o foco for maior performance nas classes de contexto (o que reduzirá a quantidade de falsos positivos na classe 0)...
# Note que por esse critério, o melhor colocado só aparece na 35 posição.
ordenacao = ['F1(Sem 0)', 'F1','F1(Só 0)','Acurácia']
df_results_all = df_results_all.sort_values(by=ordenacao, ascending=False) 
print('\tScores Finais, por', ordenacao, '(Obs: Total Perguntas Zero = %i)' % len(df_val_0[df_val_0 ['bot_id_verdade']==0]))
df_results_all.head(40)    

	Scores Finais, por ['F1(Sem 0)', 'F1', 'F1(Só 0)', 'Acurácia'] (Obs: Total Perguntas Zero = 255)


,Parâmetros,Acurácia,Precisão,Recall,F1,F1(Sem 0),F1(Só 0),Total 0,Err 0,% Err 0,Ok 0,% Ok 0
63,"{'stemmer':True, 'mpfv':1, 'mpco':0.550}",0.824,0.839,0.813,0.797,0.906,0.041,58,0,0.0,58,1.0
70,"{'stemmer':True, 'mpfv':1, 'mpco':0.520}",0.821,0.837,0.811,0.792,0.906,0.038,53,0,0.0,53,1.0
71,"{'stemmer':True, 'mpfv':1, 'mpco':0.490}",0.820,0.836,0.810,0.791,0.906,0.037,51,0,0.0,51,1.0
76,"{'stemmer':True, 'mpfv':2, 'mpco':0.790}",0.819,0.833,0.810,0.788,0.906,0.036,50,0,0.0,50,1.0
77,"{'stemmer':True, 'mpfv':2, 'mpco':0.820}",0.819,0.833,0.810,0.788,0.906,0.036,50,0,0.0,50,1.0
78,"{'stemmer':True, 'mpfv':1, 'mpco':0.460}",0.818,0.834,0.809,0.788,0.906,0.035,48,0,0.0,48,1.0
81,"{'stemmer':True, 'mpfv':2, 'mpco':0.700}",0.818,0.832,0.808,0.786,0.906,0.035,47,0,0.0,47,1.0
82,"{'stemmer':True, 'mpfv':2, 'mpco':0.730}",0.818,0.832,0.808,0.786,0.906,0.035,47,0,0.0,47,1.0
83,"{'stemmer':True, 'mpfv':2, 'mpco':0.760}",0.818,0.832,0.808,0.786,0.906,0.035,47,0,0.0,47,1.0
84,"{'stemmer':True, 'mpfv':1, 'mpco':0.430}",0.816,0.833,0.807,0.785,0.906,0.033,44,0,0.0,44,1.0


In [ ]:
ordenacao = ['F1(Sem 0)', 'F1','F1(Só 0)','Acurácia']
df_results_all = df_results_all.sort_values(by=ordenacao, ascending=False) 
print('\tScores Finais, por', ordenacao, '(Obs: Total Perguntas Zero = %i)' % len(df_val_0[df_val_0 ['bot_id_verdade']==0]))
df_results_all.head(40)  

In [84]:
print('Fim da etapa 5 e fim da solução!')

Fim da etapa 5 e fim da solução!
